## Studying the effect of pruning on untuned and tuned models - File for untuned models

### Imports and getting data from csv

In [2]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder

from sklearn.model_selection import train_test_split

import tensorflow as tf
import tensorflow_addons as tfa

from keras.layers import Dense, Conv1D, Flatten, GlobalAveragePooling1D, TimeDistributed, LSTM, AveragePooling1D, SimpleRNN, LeakyReLU

D:\Software\Anaconda\lib\site-packages\tensorflow_addons\utils\tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


In [3]:
import numpy.random as rd

In [4]:
comb_raw = pd.read_csv("SeqCombinedData.csv")
display(comb_raw)

,Usage,LaggingCurrentReactivePower,LeadingCurrentReactivePower,CO2,LaggingCurrentPowerFactor,LeadingCurrentPowerFactor,NSM,WeekStatus,Year,Month,...,SeaLevelPressure,CloudCover,Visibility,SolarRadiation,SolarEnergy,UvIndex,Conditions,SunriseHour,LoadType,SunsetHour
0,3.17,2.95,0.00,0.0,73.21,100.00,900,Weekday,2018,1,...,1026.9,3.3,9.0,139.9,12.0,6,Clear,7,Light_Load,17
1,3.28,3.56,0.00,0.0,67.76,100.00,5400,Weekday,2018,1,...,1026.9,3.3,9.0,139.9,12.0,6,Clear,7,Light_Load,17
2,3.46,4.03,0.00,0.0,65.14,100.00,9900,Weekday,2018,1,...,1026.9,3.3,9.0,139.9,12.0,6,Clear,7,Light_Load,17
3,3.89,5.00,0.00,0.0,61.40,100.00,14400,Weekday,2018,1,...,1026.9,3.3,9.0,139.9,12.0,6,Clear,7,Light_Load,17
4,3.56,4.07,0.00,0.0,65.84,100.00,18900,Weekday,2018,1,...,1026.9,3.3,9.0,139.9,12.0,6,Clear,7,Light_Load,17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7003,3.42,0.00,9.79,0.0,100.00,32.98,64800,Weekday,2018,12,...,1035.8,19.6,10.5,143.6,12.5,6,Clear,7,Light_Load,17
7004,3.96,0.00,18.29,0.0,100.00,21.16,69300,Weekday,2018,12,...,1035.8,19.6,10.5,143.6,12.5,6,Clear,7,Light_Load,17
7005,3.38,0.00,13.43,0.0,100.00,24.41,73800,Weekday,2018,12,...,1035.8,19.6,10.5,143.6,12.5,6,Clear,7,Light_Load,17
7006,3.42,0.00,13.36,0.0,100.00,24.80,78300,Weekday,2018,12,...,1035.8,19.6,10.5,143.6,12.5,6,Clear,7,Light_Load,17


In [5]:
comb_raw_filt= comb_raw[["Usage", "LaggingCurrentReactivePower", "LeadingCurrentReactivePower",
                          "CO2", "LaggingCurrentPowerFactor", "LeadingCurrentPowerFactor", "NSM",
                          "WeekStatus", "Hours", "Minutes", "IsHoliday", "Season", "Temp", "Dew",
                          "Humidity", "Precip", "PrecipProb", "PrecipCover", "PrecipType", "SnowDepth", "WindGust",
                          "WindSpeed", "WindDir", "SeaLevelPressure", "CloudCover", "Visibility", "SolarRadiation",
                          "SolarEnergy", "UvIndex", "Conditions", "SunriseHour", "LoadType", "SunsetHour"]]
load_values = comb_raw.pop("LoadType")

In [6]:
steel_cols_to_encode = ['IsHoliday','PrecipType','Season','Conditions','WeekStatus']
steel_cols_to_normalize = ['Usage', 'CO2', 'LaggingCurrentPowerFactor', 'PrecipCover', 'SnowDepth',
                          'NSM', 'Hours', 'Minutes', 'PrecipProb', 'SunriseHour']

In [7]:
comb_raw_enc = pd.get_dummies(comb_raw_filt, columns=steel_cols_to_encode)

In [8]:
steel_scaler = StandardScaler()
comb_raw_enc[steel_cols_to_normalize] = steel_scaler.fit_transform(comb_raw_enc[steel_cols_to_normalize])

In [9]:
feature_selection_columns = [
    'Usage', 'LeadingCurrentReactivePower',
       'LaggingCurrentPowerFactor', 'LeadingCurrentPowerFactor', 'NSM',
       'Hours', 'WindDir', 'SunriseHour', 'SunsetHour', 'IsHoliday_0',
       'IsHoliday_1', 'PrecipType_rain,snow', 'PrecipType_snow',
       'Season_Autumn', 'Season_Spring', 'Conditions_Rain',
       'Conditions_Snow, Partially cloudy', 'Conditions_Snow, Rain',
       'Conditions_Snow, Rain, Overcast', 'WeekStatus_Weekday'
        ]
comb_raw_filt = comb_raw_enc[feature_selection_columns]
display(comb_raw_filt)

,Usage,LeadingCurrentReactivePower,LaggingCurrentPowerFactor,LeadingCurrentPowerFactor,NSM,Hours,WindDir,SunriseHour,SunsetHour,IsHoliday_0,IsHoliday_1,"PrecipType_rain,snow",PrecipType_snow,Season_Autumn,Season_Spring,Conditions_Rain,"Conditions_Snow, Partially cloudy","Conditions_Snow, Rain","Conditions_Snow, Rain, Overcast",WeekStatus_Weekday
0,-0.722530,0.00,-0.388131,100.00,-1.678015,-1.661325,335.8,1.318323,17,0,1,0,0,0,0,0,0,0,0,1
1,-0.719255,0.00,-0.675669,100.00,-1.497584,-1.516862,335.8,1.318323,17,0,1,0,0,0,0,0,0,0,0,1
2,-0.713895,0.00,-0.813898,100.00,-1.317152,-1.372399,335.8,1.318323,17,0,1,0,0,0,0,0,0,0,0,1
3,-0.701091,0.00,-1.011218,100.00,-1.136720,-1.083473,335.8,1.318323,17,0,1,0,0,0,0,0,0,0,0,1
4,-0.710917,0.00,-0.776967,100.00,-0.956288,-0.939010,335.8,1.318323,17,0,1,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7003,-0.715086,9.79,1.025289,32.98,0.884116,0.939010,335.6,1.318323,17,1,0,0,0,0,0,0,0,0,0,1
7004,-0.699006,18.29,1.025289,21.16,1.064547,1.083473,335.6,1.318323,17,1,0,0,0,0,0,0,0,0,0,1
7005,-0.716277,13.43,1.025289,24.41,1.244979,1.227936,335.6,1.318323,17,1,0,0,0,0,0,0,0,0,0,1
7006,-0.715086,13.36,1.025289,24.80,1.425411,1.372399,335.6,1.318323,17,1,0,0,0,0,0,0,0,0,0,1


In [10]:
transformed_comb = []
for num in range(4,len(comb_raw_filt)):
    ele =comb_raw_filt[num-4:num+1]
    transformed_comb.append(ele)
transformed_comb = np.array(transformed_comb)

In [11]:
X_arr = transformed_comb
Y_arr = load_values[4:]
print(len(Y_arr))

7004


### Pruning functions

In [12]:
def weight_prune_dense_layer(k_weights1,k_weights2, b_weights, k_sparsity):
    # Copy the kernel weights and get ranked indeces of the abs
    kernel_weights1 = np.copy(k_weights1)
    ind1 = np.unravel_index(
        np.argsort(
            np.abs(kernel_weights1),
            axis=None),
        kernel_weights1.shape)
    
    kernel_weights2 = np.copy(k_weights2)
    ind2 = np.unravel_index(
        np.argsort(
            np.abs(kernel_weights2),
            axis=None),
        kernel_weights2.shape)
    
    
    # Number of indexes to set to 0
    cutoff1 = int(len(ind1[0])*k_sparsity)
    # The indexes in the 2D kernel weight matrix to set to 0
    sparse_cutoff_inds1 = (ind1[0][0:cutoff1], ind1[1][0:cutoff1])
    kernel_weights1[sparse_cutoff_inds1] = 0.
    
    cutoff2 = int(len(ind2[0])*k_sparsity)
    # The indexes in the 2D kernel weight matrix to set to 0
    sparse_cutoff_inds2 = (ind2[0][0:cutoff2], ind2[1][0:cutoff2])
    kernel_weights2[sparse_cutoff_inds2] = 0.
        
    # Copy the bias weights and get ranked indeces of the abs
    bias_weights = np.copy(b_weights)
    ind = np.unravel_index(
        np.argsort(
            np.abs(bias_weights), 
            axis=None), 
        bias_weights.shape)
        
    # Number of indexes to set to 0
    cutoff = int(len(ind[0])*k_sparsity)
    # The indexes in the 1D bias weight matrix to set to 0
    sparse_cutoff_inds = (ind[0][0:cutoff])
    bias_weights[sparse_cutoff_inds] = 0.
    
    return kernel_weights1, kernel_weights2, bias_weights

In [13]:
def sparsify_model(model, x_train, y_train, x_test, y_test, k_sparsity, pruning='weight'):

    # Copying a temporary sparse model from our original
    sparse_model = tf.keras.models.clone_model(model)
    sparse_model.set_weights(model.get_weights())
    
    # Getting a list of the names of each component (w + b) of each layer
    names = [weight.name for layer in sparse_model.layers for weight in layer.weights]
    # Getting the list of the weights for each component (w + b) of each layer
    weights = sparse_model.get_weights()
    
    # Initializing list that will contain the new sparse weights
    newWeightList = []

    # Iterate over all but the final 2 layers (the softmax)
    for i in range(0, len(weights)-2, 3):
        kernel_weights1,kernel_weights2, bias_weights = weight_prune_dense_layer(weights[i],
                                                                    weights[i+1],
                                                                    weights[i+2],
                                                                    k_sparsity)
        # Append the new weight list with our sparsified kernel weights
        newWeightList.append(kernel_weights1)
        newWeightList.append(kernel_weights2)
        
        # Append the new weight list with our sparsified bias weights
        newWeightList.append(bias_weights)

    # Adding the unchanged weights of the final 2 layers
    for i in range(len(weights)-2, len(weights)):
        unmodified_weight = np.copy(weights[i])
        newWeightList.append(unmodified_weight)

    # Setting the weights of our model to the new ones
    sparse_model.set_weights(newWeightList)
    
    # Re-compiling the Keras model (necessary for using `evaluate()`)
    sparse_model.compile(
        loss=tf.keras.losses.categorical_crossentropy,
        optimizer='adam',
        metrics=['accuracy',
                 tf.keras.metrics.Precision(),
                 tfa.metrics.F1Score(num_classes=3, average='macro', threshold=0.5)
                ])
    
    # Printing the the associated loss & Accuracy for the k% sparsity
    score = sparse_model.evaluate(x_train, y_train, verbose=0)
    score_t = sparse_model.evaluate(x_test, y_test, verbose=0)
    
    print('k% weight sparsity: ', k_sparsity,
          #'\tTrain loss: {:07.5f}'.format(score[0]),
          #'\tTrain accuracy: {:05.2f} %'.format(score[1]*100.),
          '\tTest loss: {:07.5f}'.format(score_t[0]),
          '\tTest accuracy: {:05.2f} %'.format(score_t[1]*100.),
          '\tTest precision: {:05.2f} %'.format(score_t[2]*100.),
          '\tTest F1 score: {:05.2f} %'.format(score_t[3]*100.),
         )
    
    return sparse_model, score_t

### Metrics of models over different folds of data

In [18]:
results_df = pd.DataFrame()

random_seeds = [10, 15, 20, 25, 30, 35]
k_sparsities = [0.0, 0.05, 0.10, 0.15, 0.20, 0.25, 0.30, 0.35, 0.40]
untuned_acc_array = []
untuned_precision_array=[]
untuned_f1_array=[]

count = 1
for itr in random_seeds:
    X_train, X_test, y_train, y_test = train_test_split(X_arr, Y_arr, test_size=0.20, random_state=itr)
    y_train_le = pd.get_dummies(y_train)
    y_test_le = pd.get_dummies(y_test)
    
    model_1 = tf.keras.models.Sequential()
    model_1.add(LSTM(20, return_sequences=True))
    model_1.add(LSTM(20, return_sequences=False))
    model_1.add(Dense(3, activation='softmax'))

    model_1.compile(loss='categorical_crossentropy',
              optimizer="adam",
              metrics=['accuracy',
                      tf.keras.metrics.Precision(),
                      tfa.metrics.F1Score(num_classes=3,
                                                  average='macro',
                                                  threshold=0.5)])
    
    model_1.fit(x=X_train, y=y_train_le, epochs=20)
    
    model1_loss_values = []
    model1_accs_values = []
    model1_prec_values = []
    model1_f1_values = []

    for k_sparsity in k_sparsities:
        sparse_model1, score = sparsify_model(model_1, x_train=X_train,
                                         y_train=y_train_le,
                                          x_test=X_test,
                                          y_test=y_test_le,
                                         k_sparsity=k_sparsity, 
                                         pruning='weight')
        model1_loss_values.append(score[0])
        model1_accs_values.append(score[1])
        model1_prec_values.append(score[2])
        model1_f1_values.append(score[3])
        
    col1_name = "model"+str(count)+"acc"
    col2_name = "model"+str(count)+"prec"
    col3_name = "model"+str(count)+"f1"
    count = count+1
        
    results_df[col1_name] = model1_accs_values
    results_df[col2_name] = model1_prec_values
    results_df[col3_name] = model1_f1_values

Epoch 1/20
176/176 [==============================] - 9s 10ms/step - loss: 0.9687 - accuracy: 0.5153 - precision_80: 0.6628 - f1_score: 0.2171
Epoch 2/20
176/176 [==============================] - 2s 10ms/step - loss: 0.6007 - accuracy: 0.7246 - precision_80: 0.8072 - f1_score: 0.5190
Epoch 3/20
176/176 [==============================] - 2s 10ms/step - loss: 0.4656 - accuracy: 0.7719 - precision_80: 0.8009 - f1_score: 0.6748
Epoch 4/20
176/176 [==============================] - 2s 10ms/step - loss: 0.4219 - accuracy: 0.7967 - precision_80: 0.8143 - f1_score: 0.7122
Epoch 5/20
176/176 [==============================] - 2s 10ms/step - loss: 0.3916 - accuracy: 0.8215 - precision_80: 0.8376 - f1_score: 0.7559
Epoch 6/20
176/176 [==============================] - 2s 10ms/step - loss: 0.3842 - accuracy: 0.8251 - precision_80: 0.8362 - f1_score: 0.7658
Epoch 7/20
176/176 [==============================] - 2s 10ms/step - loss: 0.3631 - accuracy: 0.8317 - precision_80: 0.8450 - f1_score: 0.7784

176/176 [==============================] - 2s 10ms/step - loss: 0.3820 - accuracy: 0.8324 - precision_120: 0.8537 - f1_score: 0.7637
Epoch 6/20
176/176 [==============================] - 2s 10ms/step - loss: 0.3563 - accuracy: 0.8451 - precision_120: 0.8595 - f1_score: 0.7910
Epoch 7/20
176/176 [==============================] - 2s 10ms/step - loss: 0.3417 - accuracy: 0.8479 - precision_120: 0.8602 - f1_score: 0.7973
Epoch 8/20
176/176 [==============================] - 2s 9ms/step - loss: 0.3301 - accuracy: 0.8528 - precision_120: 0.8635 - f1_score: 0.8038
Epoch 9/20
176/176 [==============================] - 2s 10ms/step - loss: 0.3138 - accuracy: 0.8579 - precision_120: 0.8665 - f1_score: 0.8100
Epoch 10/20
176/176 [==============================] - 2s 10ms/step - loss: 0.3147 - accuracy: 0.8553 - precision_120: 0.8661 - f1_score: 0.8108
Epoch 11/20
176/176 [==============================] - 2s 10ms/step - loss: 0.2999 - accuracy: 0.8667 - precision_120: 0.8753 - f1_score: 0.8254
Ep

In [19]:
display(results_df)

,model1acc,model1prec,model1f1,model2acc,model2prec,model2f1,model3acc,model3prec,model3f1,model4acc,model4prec,model4f1,model5acc,model5prec,model5f1,model6acc,model6prec,model6f1
0,0.858469,0.867345,0.815639,0.892736,0.906456,0.861495,0.907728,0.914265,0.884877,0.896306,0.903068,0.863371,0.898804,0.904495,0.869921,0.896484,0.906968,0.867523
1,0.771551,0.779612,0.731420,0.882920,0.897215,0.858633,0.892022,0.906002,0.865901,0.906479,0.911509,0.876279,0.892379,0.897764,0.863962,0.900589,0.911770,0.870540
2,0.620917,0.630821,0.465904,0.864537,0.877892,0.827648,0.855435,0.872835,0.836981,0.669641,0.679846,0.547420,0.885597,0.890354,0.851442,0.851508,0.862551,0.802436
3,0.583973,0.596535,0.427747,0.840086,0.858473,0.792843,0.833839,0.842500,0.777350,0.690880,0.708435,0.577265,0.884169,0.889795,0.851071,0.699804,0.717866,0.575719
4,0.579154,0.587050,0.384205,0.837052,0.852810,0.781231,0.521149,0.521677,0.242842,0.672140,0.686348,0.548406,0.858112,0.869813,0.812210,0.693379,0.709665,0.563258
5,0.522756,0.523154,0.256631,0.793503,0.811657,0.686598,0.523648,0.524726,0.246221,0.754060,0.772183,0.727128,0.849188,0.858341,0.806328,0.654114,0.670960,0.500901
6,0.658576,0.665753,0.478660,0.515795,0.515795,0.227666,0.529002,0.530726,0.259709,0.746386,0.755956,0.668150,0.518829,0.519014,0.227786,0.624130,0.636687,0.453713
7,0.514724,0.515661,0.238582,0.515617,0.515617,0.227208,0.515974,0.516262,0.230839,0.712297,0.716856,0.573023,0.518829,0.518829,0.227732,0.530430,0.532049,0.255857
8,0.516688,0.518805,0.241406,0.515795,0.515979,0.227719,0.519900,0.521724,0.239614,0.697305,0.702239,0.505923,0.519543,0.519636,0.229576,0.598965,0.607499,0.410954


In [20]:
#Save results to csv for later use
results_df.to_csv("UntunedPruningResults.csv", index=True)